<a href="https://colab.research.google.com/github/vishwanaik15/DM-ASSIGNMENT03/blob/main/Assignment_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Vishwa Chetankumar Naik</br>
UTA ID:1001871311</br>
Assignment - 03 </br>


**Import Libraries**

In [949]:
import os
import glob
import numpy as np
import pandas as pd
from pandas import Series
import re
from sys import path
from collections import Counter
import string
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import re


**A)Divide the dataset as train and development.**</br>
--Here I have used dataset from kaggle sentiment labelled sentences dataset.</br>
--I have imported test data at the last of  assignment because in the last question it is required, so just to remove confusion.




In [950]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [951]:
imdb_data = pd.read_csv('/content/gdrive/My Drive/data/imdb_labelled.txt', delimiter='\t', header=None, names=['review', 'sentiment'])
print(imdb_data.shape)

(748, 2)


#dataset understanding and preprocess

In [952]:
imdb_data.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [953]:
imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     748 non-null    object
 1   sentiment  748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [954]:
imdb_data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [955]:
imdb_data.sentiment.value_counts()

1    386
0    362
Name: sentiment, dtype: int64

In [956]:
shuffle = imdb_data.sample(frac=1).to_numpy()
#shuffling data using sample method to get random values in train and development set each time 
size = int(0.60*len(imdb_data))
#i have taken 60% of data as training and rest of 40% data as development set 
train_set = shuffle[:size]
development_set = shuffle[size:]
print("Size of training set: ",len(train_set))
print("Shape of training set:",{train_set.shape})
print("Size of develpment set: ",len(development_set))
print("Shape of development set:",{development_set.shape})


Size of training set:  448
Shape of training set: {(448, 2)}
Size of develpment set:  300
Shape of development set: {(300, 2)}


In [957]:
train_set

array([['My rating: just 3 out of 10.  ', 0],
       ['The movie in movie situations in the beginning and through the game that is played with her by the "acting coach" are fascinating.  ',
        1],
       ['I rather enjoyed it.  ', 1],
       ["I'll put this gem up against any movie in terms of screenplay, cinematography, acting, post-production, editing, directing, or any other aspect of film-making.  ",
        1],
       ["It's one of the movies I need to see whenever it comes on TV...never mind the fact that I already have it memorized!  ",
        1],
       ['Every element of this story was so over the top, excessively phony and contrived that it was painful to sit through.  ',
        0],
       ['Both do good jobs and are quite amusing.  ', 1],
       ["The film's dialogue is natural, real to life.  ", 1],
       ['The aerial scenes were well-done.  ', 1],
       ['It is just the sort of pap that is screened in the afternoon to punish the unemployed for not having jobs.  ',

In [958]:
development_set

array([['Too politically correct.  ', 0],
       ["I've seen soap operas more intelligent than this movie.  ", 0],
       ['I knew when I saw the film that more great things were to come from this gifted actor.  ',
        1],
       ['This is one I did not have, but I watched it recently on Fox Movie Channel, and was very disappointed.  ',
        0],
       ['It was horrendous.  ', 0],
       ['The telephone repair man was really funny and his reactions to the nuts and bitchy boss were truly genuine.  ',
        1],
       ['Again, no plot at all.  ', 0],
       ['The film gives meaning to the phrase, "Never in the history of human conflict has so much been owed by so many to so few.  ',
        1],
       ['Maybe there would be a reasonable explanation for this atrocity.  ',
        0],
       ["Clever and a real crowd-pleaser--this film still ranks among Mickey's best films even after 80 wonderful years.  ",
        1],
       ["This is a bad film, with bad writing, and good actors

In [959]:
X_train = train_set[:,0] #reviews in X_train
Y_train = train_set[:,-1] #last column goes in Y_train which shows labels 0 or 1
X_development = development_set[:,0]
Y_development = development_set[:,-1]

In [960]:
X_train


array(['My rating: just 3 out of 10.  ',
       'The movie in movie situations in the beginning and through the game that is played with her by the "acting coach" are fascinating.  ',
       'I rather enjoyed it.  ',
       "I'll put this gem up against any movie in terms of screenplay, cinematography, acting, post-production, editing, directing, or any other aspect of film-making.  ",
       "It's one of the movies I need to see whenever it comes on TV...never mind the fact that I already have it memorized!  ",
       'Every element of this story was so over the top, excessively phony and contrived that it was painful to sit through.  ',
       'Both do good jobs and are quite amusing.  ',
       "The film's dialogue is natural, real to life.  ",
       'The aerial scenes were well-done.  ',
       'It is just the sort of pap that is screened in the afternoon to punish the unemployed for not having jobs.  ',
       "You'd have to have the IQ of particularly stupid mollusk not to see t

**b>Build a vocabulary as list. **</br>
[‘the’ ‘I’ ‘happy’ … ] </br>
You may omit rare words for example if the occurrence is less than five times
A reverse index as the key value might be handy</br>
{“the”: 0, “I”:1, “happy”:2 , … }


In [961]:
#vocabulary list
a = {}
vocab_list=[]
def build_vocab_list(imdb_data):
    a = counting_number_of_unique_words(imdb_data)
    for i in a.keys():
        vocab_list.append(i)
    return vocab_list

In [962]:
#counting unique number of words in dataset so we can compare after removing less occurring words 
words_count = {}
def counting_number_of_unique_words(imdb_data):
    for line in imdb_data:
        for word in line.split():
            if word in words_count:
                words_count[word] = words_count[word] + 1
            else:
                words_count[word] = 1
    return words_count


In [963]:
#before removing less occurring data 
words_count = counting_number_of_unique_words(X_train)
print("Total unique words in the given dataset before removing less occurring words: ",len(words_count))


Total unique words in the given dataset before removing less occurring words:  3066


In [964]:
print(words_count) #with less occurring words/


{'My': 1, 'rating:': 1, 'just': 35, '3': 1, 'out': 22, 'of': 251, '10.': 2, 'The': 130, 'movie': 91, 'in': 117, 'situations': 1, 'the': 418, 'beginning': 1, 'and': 281, 'through': 5, 'game': 1, 'that': 94, 'is': 211, 'played': 6, 'with': 53, 'her': 14, 'by': 31, '"acting': 1, 'coach"': 1, 'are': 49, 'fascinating.': 1, 'I': 172, 'rather': 3, 'enjoyed': 6, 'it.': 21, "I'll": 3, 'put': 2, 'this': 132, 'gem': 2, 'up': 9, 'against': 1, 'any': 17, 'terms': 1, 'screenplay,': 1, 'cinematography,': 1, 'acting,': 5, 'post-production,': 1, 'editing,': 1, 'directing,': 1, 'or': 20, 'other': 9, 'aspect': 1, 'film-making.': 1, "It's": 21, 'one': 39, 'movies': 14, 'need': 1, 'to': 149, 'see': 25, 'whenever': 1, 'it': 79, 'comes': 1, 'on': 44, 'TV...never': 1, 'mind': 1, 'fact': 3, 'already': 1, 'have': 33, 'memorized!': 1, 'Every': 3, 'element': 2, 'story': 17, 'was': 107, 'so': 29, 'over': 7, 'top,': 1, 'excessively': 1, 'phony': 1, 'contrived': 1, 'painful': 1, 'sit': 1, 'through.': 1, 'Both': 2, '

In [965]:
#cleandata
#reomving less frequent words from dataset
repeating_words = []
for i in words_count.keys():
    if (words_count[i]<5):
          repeating_words.append(i)
for i in repeating_words:
    del words_count[i]
len(words_count)
def removing_less_occurring_words(imdb_data,words_count):
    for i, review in enumerate(imdb_data):
        clean = []
        for word in review.split():
            if word in words_count.keys():
                clean.append(word)
        clean = ' '.join(clean)
        imdb_data[i] = clean
    return imdb_data

In [966]:
#after removing less occurring data from vocab_list and so from X_train
X_train = removing_less_occurring_words(X_train,words_count)
print("Total unique words in the given dataset after removing less occurring words: ",len(X_train))
print('')
#print("Length of words dictionary after removing less occurred data: ",len(words_count))

Total unique words in the given dataset after removing less occurring words:  448



In [967]:
#my words dictionary after removing less occurring words
print(words_count)


{'just': 35, 'out': 22, 'of': 251, 'The': 130, 'movie': 91, 'in': 117, 'the': 418, 'and': 281, 'through': 5, 'that': 94, 'is': 211, 'played': 6, 'with': 53, 'her': 14, 'by': 31, 'are': 49, 'I': 172, 'enjoyed': 6, 'it.': 21, 'this': 132, 'up': 9, 'any': 17, 'acting,': 5, 'or': 20, 'other': 9, "It's": 21, 'one': 39, 'movies': 14, 'to': 149, 'see': 25, 'it': 79, 'on': 44, 'have': 33, 'story': 17, 'was': 107, 'so': 29, 'over': 7, 'do': 10, 'good': 31, 'dialogue': 6, 'real': 14, 'scenes': 10, 'were': 17, 'It': 36, 'for': 56, 'not': 42, 'could': 13, 'be': 34, 'as': 58, 'she': 7, 'off': 6, 'between': 8, 'how': 19, 'little': 9, 'there': 18, 'two': 9, 'got': 6, 'a': 270, 'big': 5, 'my': 25, 'So': 5, 'Even': 6, 'way.': 5, 'great': 22, 'part': 5, 'If': 12, 'you': 34, 'your': 12, 'end': 6, 'all': 25, 'must': 5, 'waste': 9, 'time.': 10, "didn't": 13, 'make': 14, 'boring': 6, 'movie.': 18, 'totally': 8, 'acting': 22, 'In': 8, 'short': 5, '-': 14, 'time': 16, 'would': 18, 'recommend': 8, 'anyone': 11

In [968]:
review_0_train = train_set[Y_train == 0]
print(f'Total 0 (negative) review in train dataset : {len(review_0_train)}')

review_1_train = train_set[Y_train == 1]
print(f'Total 1 (positive) review in train dataset : {len(review_1_train)}')

Total 0 (negative) review in train dataset : 216
Total 1 (positive) review in train dataset : 232


c.) Find basic probability gor positive and negative.

In [969]:
#count basic probability for positive and negative reviews
def calculate_prior_class_prob(Y_train):
  prior_class_prob = {}
  for c in np.unique(Y_train):
    count = sum(Y_train == c)
    prior_class_prob[c] = count / Y_train.size
  return prior_class_prob

prior_class_prob = calculate_prior_class_prob(Y_train)
print(f'prior probability of class 0 (negative) : {prior_class_prob[0]}')
print(f'prior probability of class 1 (positive) : {prior_class_prob[1]}')

prior probability of class 0 (negative) : 0.48214285714285715
prior probability of class 1 (positive) : 0.5178571428571429


In [970]:
#here we are making dictionary containing word and it's occurrance in data
def total_words_in_data(imdb_data, words_count):
    new_dict = {}
    for word in words_count.keys():
        count = 0
        for review in imdb_data:
            if word in review:
                count = count + 1
        new_dict[word] = count
    return new_dict

In [971]:
words_occ = total_words_in_data(X_train,words_count)
print("Number of word the in the data : ",words_occ['the'])

Number of word the in the data :  218


**-->Probability of the occurrence**</br>
P[“the”] = num of documents containing ‘the’ / num of all documents


In [972]:
#probability of given word in dataset
def prob_calc_of_word_in_data(imdb_data, word):
    count = 0
    if word not in words_occ.keys():
        count = 0
    else:
        count = words_occ[word]
    return count/len(imdb_data)

In [973]:
prob_word_in_data = prob_calc_of_word_in_data(X_train, 'the')
print("Probability of word in whole document: p[the] : ",prob_word_in_data*100)

Probability of word in whole document: p[the] :  48.660714285714285


**-->Conditional probability based on the sentiment</br>**
P[“the” | Positive]  = # of positive documents containing “the”  


In [974]:
def positive_words_count(imdb_data, label, words_count):
    positive_words = {}
    for word in words_count.keys():
        pos_count = 0
        for index, review in enumerate(imdb_data):
            if word in review.split() and label[index]==1:
                pos_count = pos_count + 1
        positive_words[word] = pos_count
    return positive_words

def negative_words_count(imdb_data, label, words_count):
    negative_words = {}
    for word in words_count.keys():
        neg_count = 0
        for index, review in enumerate(imdb_data):
            if word in review.split() and label[index]==0:
                neg_count = neg_count + 1
        negative_words[word] = neg_count
    return negative_words
positive_words = positive_words_count(X_train, Y_train, words_count)
negative_words = negative_words_count(X_train, Y_train, words_count)

In [975]:
#in how many positive documents word "the" comes
positive_words['the']

110

In [976]:
#in how many negative documents word "the" comes
negative_words['the']

89

In [977]:
def positive_doc_count(imdb_data):
    positive_doc = 0
    for i in range(len(imdb_data)):
        if (imdb_data[i]==1):
            positive_doc = positive_doc + 1
    return positive_doc
def negative_doc_count(imdb_data):
    negative_doc = 0
    for i in range(len(imdb_data)):
        if (imdb_data[i]==0):
            negative_doc = negative_doc + 1
    return negative_doc
def pos_neg_doc_count(imdb_data, attribute):
    if attribute == "positive":
        cnt = positive_doc_count(imdb_data)
    else:
        cnt = negative_doc_count(imdb_data)
    return cnt
positive_doc = pos_neg_doc_count(Y_train,attribute = "positive")
negative_doc = pos_neg_doc_count(Y_train,attribute = "negative")

In [978]:
#number of positive documents
print(positive_doc)
print(negative_doc)

232
216


In [979]:
#probability of given word in positive or negative document
def num_of_word_in_posneg(word,words_count, attribute):
    count = 0
    if attribute =="positive":
        if word in positive_words.keys():
            count = positive_words[word]
        else:
            count = 0
            
    elif attribute=="negative":
        if word in negative_words.keys():
            count = negative_words[word]
        else:
            count = 0
    return count

In [980]:
numerator = num_of_word_in_posneg('the',words_count, attribute = "positive")
p = numerator/positive_doc
print("p[the|positive]: "+str(p*100)+" %")

p[the|positive]: 47.41379310344828 %


In [981]:
numerator = num_of_word_in_posneg('the',words_count, attribute = "negative")
p = numerator/positive_doc
print("p[the|negative]: "+str(p*100)+" %")

p[the|negative]: 38.362068965517246 %


In [982]:
#probability for p[positive|the] without NAIVE BAYES
prob = (p*(positive_doc/len(X_train)))/prob_word_in_data
print("Probability of given word in data",prob*100)

Probability of given word in data 40.8256880733945


**D) Calculate accuracy using dev dataset **


In [983]:
def NaiveBayes(imdb_data, label, word,words_count, attribute):
    numerator = num_of_word_in_posneg(word, words_count,attribute)
    denominator = prob_calc_of_word_in_data(imdb_data, word)
    posnegdoc = pos_neg_doc_count(label,attribute)
    if denominator == 0:
        denominator = 0.1
    else:
        denominator = denominator
    result = (numerator/posnegdoc)*(posnegdoc/len(imdb_data))/denominator
    return result

In [984]:
def NaiveBayes_for_data(imdb_data,label,words_count,data1):
    positive_pred = 1
    negative_pred = 1
    y_pred = []
    for index,review in enumerate(data1):
        for word in review.split(" "):
            a = NaiveBayes(imdb_data, label, word,words_count, attribute = "negative")
            if a == 0:
                a = 0.1
            else:
                a = a
            negative_pred = negative_pred * a
            #print("n",negative_pred)
            b = NaiveBayes(imdb_data, label, word,words_count, attribute = "positive")
            if b == 0:
                b = 0.1
            else:
                b = b
            positive_pred = positive_pred * b
            #print("p",positive_pred)
        if positive_pred > negative_pred:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

In [985]:
y_pred = NaiveBayes_for_data(X_train,Y_train,words_count,X_development)

In [986]:
def accuracy(label, prediction):
    pred = 0
    for i in range(len(label)):
        if(label[i]==prediction[i]):
            pred = pred + 1
        acc = float(pred/len(label))
    return acc

 **E)Do following experiments
Compare the effect of Smoothing**



In [987]:
#Accuracy without smoothing
Accuracy = accuracy(Y_development,y_pred)
print("Accuracy without smoothing the data:  "+str(Accuracy*100)+" %")

Accuracy without smoothing the data:  50.33333333333333 %


In [988]:
#smoothing the data
def NaiveBayesSmooth(imdb_data, label, word,words_count, attribute):
    numerator = num_of_word_in_posneg(word, words_count, attribute)
    denominator = prob_calc_of_word_in_data(imdb_data, word)
    posnegdoc = pos_neg_doc_count(label,attribute)
    if denominator == 0:
        denominator = 0.1
    else:
        denominator = denominator
    result = (((numerator/posnegdoc)*(posnegdoc/len(imdb_data)))+1)/(denominator+2)
    return result
def NaiveBayes_for_Smoothdata(imdb_data,label,words_count,data1):
    positive_pred = 1
    negative_pred = 1
    y_pred = []
    for index,review in enumerate(data1):
        for word in review.split(" "):
            a = NaiveBayesSmooth(imdb_data, label, word,words_count, attribute = "negative")
            if a == 0:
                a = 0.1
            else:
                a = a
            negative_pred = negative_pred * a
            #print("n",negative_pred)
            b = NaiveBayesSmooth(imdb_data, label, word,words_count, attribute = "positive")
            if b == 0:
                b = 0.1
            else:
                b = b
            positive_pred = positive_pred * b
            #print("p",positive_pred)
        if positive_pred > negative_pred:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

In [989]:
#predicted values for given data 
y_pred = NaiveBayes_for_Smoothdata(X_train,Y_train,words_count,X_development)

In [990]:
#accuracy after smoothing the data
Accuracy = accuracy(Y_development,y_pred)
print("Accuracy after smoothing the data:  "+str(Accuracy*100)+" %")

Accuracy after smoothing the data:  50.66666666666667 %


**-->Derive Top 10 words that predicts positive and negative class
P[Positive| word]**

In [991]:
import operator,itertools
top_positive={}
for word in words_count.keys():
    acc = NaiveBayes(X_train, Y_train, word,words_count, attribute="positive")
    top_positive[word] = acc
top_reversed = dict(sorted(top_positive.items(),key=operator.itemgetter(1),reverse=True))
top_10_positive = dict(itertools.islice(top_reversed.items(), 10))

In [992]:
#top10 positive 
print("Top 10 words that predict positive class\n",top_10_positive.keys())

Top 10 words that predict positive class
 dict_keys(['their', 'it!', 'well.', 'John', 'interesting', 'wonderful', 'people', 'worth', 'played', 'job'])


In [993]:
top_negative={}
for word in words_count.keys():
    acc = NaiveBayes(X_train, Y_train, word,words_count, attribute="negative")
    top_negative[word] = acc
top_reversed = dict(sorted(top_positive.items(),key=operator.itemgetter(1),reverse=False))
top_10_negative = dict(itertools.islice(top_reversed.items(), 10))

In [994]:
top_negative={}
for word in words_count.keys():
    acc = NaiveBayes(X_train, Y_train, word,words_count, attribute="negative")
    top_negative[word] = acc
top_reversed = dict(sorted(top_positive.items(),key=operator.itemgetter(1),reverse=False))
top_10_negative = dict(itertools.islice(top_reversed.items(), 10))

**F)Using the test dataset
Use the optimal hyperparameters you found in the step e, and use it to calculate the final accuracy.**

In [995]:
shuffle = imdb_data.sample(frac=1).to_numpy()
#shuffling data using sample method to get random values in train and development set each time 
size = int(0.40*len(imdb_data))
#i have taken 40% of data as testing.
test_set = shuffle[:size]

print("Size of test set: ",len(test_set))
print("Shape of test set:",{test_set.shape})



Size of test set:  299
Shape of test set: {(299, 2)}


In [996]:
X_test = test_set[:,0] #reviews in X_train
Y_test = test_set[:,-1]

In [997]:
y_pred = NaiveBayes_for_data(X_train,Y_train,words_count,X_test)

In [998]:
#accuracy without smoothing the test data
Accuracy = accuracy(Y_test,y_pred)
print("Accuracy without smoothing the TEST data:  "+str(Accuracy*100)+" %")

Accuracy without smoothing the TEST data:  47.82608695652174 %


In [999]:
y_pred = NaiveBayes_for_Smoothdata(X_train,Y_train,words_count,X_test)

In [1000]:
#accuracy after smoothing the test data
Accuracy = accuracy(Y_test,y_pred)
print("Accuracy after smoothing the TEST data:  "+str(Accuracy*100)+" %")

Accuracy after smoothing the TEST data:  53.84615384615385 %
